## Setup

In [1]:
import os
import requests
import json
from pathlib import Path

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [2]:
load_eoepca_state()

In [ ]:
platform_domain = os.environ.get("INGRESS_HOST")
app_quality_domain = f'https://application-quality.{platform_domain}'
oidc_client_id = "application-quality"

username = os.environ.get("KEYCLOAK_TEST_USER")
password = os.environ.get("KEYCLOAK_TEST_PASSWORD")

Resource Health URL: https://resource-health.notebook-test.develop.eoepca.org


## Authenticate with EOEPCA IAM

In [ ]:
username = os.environ.get("KEYCLOAK_TEST_USER")
password = os.environ.get("KEYCLOAK_TEST_PASSWORD")

access_token = get_access_token(username, password, oidc_client_id)
headers = {"Authorization": f"Bearer {access_token}"}

if access_token:
    print("✅ Successfully authenticated.")
else:
    print("❌ Authentication failed.")

## Retrieve Available Pipelines

In [ ]:
pipelines_url = f"{app_quality_domain}/api/pipelines"
response = requests.get(pipelines_url, headers=headers)

if response.status_code == 200:
    pipelines = response.json()
    print("✅ Available Pipelines:")
    for pipeline in pipelines:
        print(f"- {pipeline['name']} (ID: {pipeline['id']})")
else:
    print(f"❌ Failed to retrieve pipelines. Status code: {response.status_code}")

## Execute a Pipeline

In [ ]:
if pipelines:
    selected_pipeline_id = pipelines[0]['id']
    execute_url = f"{pipelines_url}/{selected_pipeline_id}/execute"

    execution_payload = {
        "git_url": "https://github.com/example/repo.git",
        "git_branch": "main"
    }

    response = requests.post(execute_url, headers={**headers, "Content-Type": "application/json"}, json=execution_payload)

    if response.status_code in [200, 201]:
        execution_id = response.json().get("execution_id")
        print(f"✅ Pipeline execution initiated. Execution ID: {execution_id}")
    else:
        print(f"❌ Failed to execute pipeline. Response: {response.text}")
else:
    print("❌ No pipelines available for execution.")

## Monitor Pipeline Execution

In [ ]:
if execution_id:
    execution_status_url = f"{app_quality_domain}/api/executions/{execution_id}/status"
    response = requests.get(execution_status_url, headers=headers)

    if response.status_code == 200:
        status_info = response.json()
        print(f"✅ Pipeline Execution Status:")
        print(json.dumps(status_info, indent=2))
    else:
        print(f"❌ Failed to retrieve execution status. Status code: {response.status_code}")
else:
    print("⚠️ No execution ID available to monitor.")

## Retrieve and Display Execution Reports

In [ ]:
if execution_id:
    reports_url = f"{app_quality_domain}/api/executions/{execution_id}/reports"
    response = requests.get(reports_url, headers=headers)

    if response.status_code == 200:
        reports = response.json()
        print(f"✅ Execution Reports:")
        for report in reports:
            tool_name = report.get("tool_name", "Unnamed Tool")
            findings = report.get("findings", [])
            print(f"\nTool: {tool_name}, Findings Count: {len(findings)}")
            for finding in findings[:5]:  # Display only first 5 findings
                print(f"- {finding.get('message', 'No detail available')}")
    else:
        print(f"❌ Failed to retrieve reports. Status code: {response.status_code}")
else:
    print("⚠️ No execution ID available for retrieving reports.")